In [3]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BDAS_jlin897').getOrCreate()

In [4]:
# read all the csv file
#all_csv_path = '*.csv'
#accident =  spark.read.format('csv')\
#   .option('header', 'true')\
#    .option('inferSchema', 'true')\
#    .load(all_csv_path)


accident = spark.read.csv("accidents_2012_to_2014.csv", header=True, inferSchema= True) 

In [5]:
print(accident.count())
print(accident.columns)
print(accident.dtypes)


464697
['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week', 'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)', '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control', '2nd_Road_Class', '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident', 'LSOA_of_Accident_Location', 'Year']
[('Accident_Index', 'string'), ('Location_Easting_OSGR', 'int'), ('Location_Northing_OSGR', 'int'), ('Longitude', 'double'), ('Latitude', 'double'), ('Police_Force', 'int'), ('Accident_Severity', 'int'), ('Number_of_Vehicles', 'int'), ('Number_of_Casualties', 'int'), ('Date'

In [6]:
print(accident[["Accident_Severity"]].describe().show())


+-------+-------------------+
|summary|  Accident_Severity|
+-------+-------------------+
|  count|             464697|
|   mean| 2.8334613737553718|
| stddev|0.40202902164685816|
|    min|                  1|
|    max|                  3|
+-------+-------------------+

None


In [7]:
print(accident[["Day_of_Week"]].describe().show())

+-------+------------------+
|summary|       Day_of_Week|
+-------+------------------+
|  count|            464697|
|   mean| 4.108739673378567|
| stddev|1.9164285644856303|
|    min|                 1|
|    max|                 7|
+-------+------------------+

None


In [8]:
import pandas as pd
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)

pd.set_option('display.max_rows', None)
accident = pd.read_csv("accidents_2012_to_2014.csv",low_memory=False)


In [9]:
print("Columns in this data sets are: ")
print(accident.columns)

Columns in this data sets are: 
Index(['Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR',
       'Longitude', 'Latitude', 'Police_Force', 'Accident_Severity',
       'Number_of_Vehicles', 'Number_of_Casualties', 'Date', 'Day_of_Week',
       'Time', 'Local_Authority_(District)', 'Local_Authority_(Highway)',
       '1st_Road_Class', '1st_Road_Number', 'Road_Type', 'Speed_limit',
       'Junction_Detail', 'Junction_Control', '2nd_Road_Class',
       '2nd_Road_Number', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location', 'Year'],
      dtype='object')


In [10]:
accident_missing = accident.isnull().sum() * 100 / len(accident)
accident_missing_value_df = pd.DataFrame({'column_name': accident.columns,'percent_missing':accident_missing})
print(accident_missing_value_df)

accident_current = accident[accident["Road_Surface_Conditions"].notnull() &
                            (accident["Weather_Conditions"].notnull())]
print("The dimension of the traffic accident 2012 to 2014 dataset is "+ str(accident_current.shape[0]) + 
      " rows and " + str(accident_current.shape[1]) + " columns.")

                                                                             column_name  \
Accident_Index                                                            Accident_Index   
Location_Easting_OSGR                                              Location_Easting_OSGR   
Location_Northing_OSGR                                            Location_Northing_OSGR   
Longitude                                                                      Longitude   
Latitude                                                                        Latitude   
Police_Force                                                                Police_Force   
Accident_Severity                                                      Accident_Severity   
Number_of_Vehicles                                                    Number_of_Vehicles   
Number_of_Casualties                                                Number_of_Casualties   
Date                                                                            

In [11]:
accident_current.groupby(by="Road_Surface_Conditions").size()
print(accident_current["Road_Surface_Conditions"].isnull().sum(axis=0))

0


In [12]:
accident_current.groupby(by="Weather_Conditions").size()
print(accident_current["Weather_Conditions"].isnull().sum(axis=0))

0


In [13]:
accident_current_missing = accident_current.isnull().sum() * 100 / len(accident_current)
accident_missing_value_df = pd.DataFrame({'column_name': accident_current.columns,'percent_missing':accident_current_missing})
print(accident_missing_value_df)



                                                                             column_name  \
Accident_Index                                                            Accident_Index   
Location_Easting_OSGR                                              Location_Easting_OSGR   
Location_Northing_OSGR                                            Location_Northing_OSGR   
Longitude                                                                      Longitude   
Latitude                                                                        Latitude   
Police_Force                                                                Police_Force   
Accident_Severity                                                      Accident_Severity   
Number_of_Vehicles                                                    Number_of_Vehicles   
Number_of_Casualties                                                Number_of_Casualties   
Date                                                                            

In [27]:
print(accident_current.groupby(by="Junction_Control").size())
print(accident_current["Junction_Control"].isnull().sum(axis=0))

Junction_Control
Authorised person              676
Automatic traffic signal     50124
Giveway or uncontrolled     232549
Stop Sign                     2286
dtype: int64
178307


In [12]:
valid_cols = ["Police_Force","Accident_Severity","Number_of_Vehicles","Number_of_Casualties",
             "Date","Day_of_Week","Time","Road_Type","Speed_limit","Light_Conditions",
             "Weather_Conditions","Road_Surface_Conditions","Urban_or_Rural_Area"]


accident_current = accident_current[valid_cols]
print(accident_current.columns)
accident_current_missing = accident_current.isnull().sum() * 100 / len(accident_current)
accident_missing_value_df = pd.DataFrame({'column_name': accident_current.columns,'percent_missing':accident_current_missing})
print(accident_missing_value_df)

Index(['Police_Force', 'Accident_Severity', 'Number_of_Vehicles',
       'Number_of_Casualties', 'Date', 'Day_of_Week', 'Time', 'Road_Type',
       'Speed_limit', 'Light_Conditions', 'Weather_Conditions',
       'Road_Surface_Conditions', 'Urban_or_Rural_Area'],
      dtype='object')
                                     column_name  percent_missing
Police_Force                        Police_Force         0.000000
Accident_Severity              Accident_Severity         0.000000
Number_of_Vehicles            Number_of_Vehicles         0.000000
Number_of_Casualties        Number_of_Casualties         0.000000
Date                                        Date         0.000000
Day_of_Week                          Day_of_Week         0.000000
Time                                        Time         0.002802
Road_Type                              Road_Type         0.000000
Speed_limit                          Speed_limit         0.000000
Light_Conditions                Light_Conditions       

In [13]:
print(accident["Time"].isnull().sum(axis=0))
accident_current.dropna(inplace=True)
accident_current["Time"].isnull().sum(axis=0)
print(accident_current.shape)

13
(463929, 13)


In [14]:
accident_current.groupby(by="Weather_Conditions").count()

,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Road_Type,Speed_limit,Light_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area
Weather_Conditions,,,,,,,,,,,,
Fine with high winds,5008,5008,5008,5008,5008,5008,5008,5008,5008,5008,5008,5008
Fine without high winds,372985,372985,372985,372985,372985,372985,372985,372985,372985,372985,372985,372985
Fog or mist,2408,2408,2408,2408,2408,2408,2408,2408,2408,2408,2408,2408
Other,8247,8247,8247,8247,8247,8247,8247,8247,8247,8247,8247,8247
Raining with high winds,7119,7119,7119,7119,7119,7119,7119,7119,7119,7119,7119,7119
Raining without high winds,57054,57054,57054,57054,57054,57054,57054,57054,57054,57054,57054,57054
Snowing with high winds,733,733,733,733,733,733,733,733,733,733,733,733
Snowing without high winds,2707,2707,2707,2707,2707,2707,2707,2707,2707,2707,2707,2707
Unknown,7668,7668,7668,7668,7668,7668,7668,7668,7668,7668,7668,7668


In [15]:
# select rows
accident_current = accident_current[accident_current["Weather_Conditions"] != "Other"]
accident_current = accident_current[accident_current["Weather_Conditions"] != "Unknown"]
print(accident_current.shape)
print(accident_current.isnull().sum(axis=0))

(448014, 13)
Police_Force               0
Accident_Severity          0
Number_of_Vehicles         0
Number_of_Casualties       0
Date                       0
Day_of_Week                0
Time                       0
Road_Type                  0
Speed_limit                0
Light_Conditions           0
Weather_Conditions         0
Road_Surface_Conditions    0
Urban_or_Rural_Area        0
dtype: int64


In [16]:
# construct data
import matplotlib.pyplot as plt

accident_current["Weekend"] = accident_current["Day_of_Week"] == 1 | (accident_current["Day_of_Week"] ==7)
print(accident_current["Weekend"].value_counts())


False    399498
True      48516
Name: Weekend, dtype: int64


In [17]:
accident_current["Night"] = pd.to_datetime(accident_current["Time"]).dt.hour <= 6 |(pd.to_datetime(accident_current["Time"]).dt.hour >=19)
print(accident_current["Night"].value_counts())

False    437314
True      10700
Name: Night, dtype: int64


In [18]:
accident_current['Severity1'] = 0
accident_current.loc[accident_current['Accident_Severity'] == 1, 'Severity1'] = 1
print(accident_current.Severity1.value_counts())
accident_current = accident_current.drop(['Accident_Severity'], axis = 1)

accident_current = pd.concat([accident_current[accident_current['Severity1']==1].sample(10000, replace = True),
                             accident_current[accident_current['Severity1']==0].sample(10000,replace=False)], axis=0)
print('resampled data:', accident_current.Severity1.value_counts())


0    442827
1      5187
Name: Severity1, dtype: int64
resampled data: 1    10000
0    10000
Name: Severity1, dtype: int64


In [35]:
columns = accident_current.columns.tolist()
columns = [c for c in columns if c not in["Accident_Severity","Police_Force","Number_of_Vehicles",
                                          "Number_of_Casualties","Date","Day_of_Week","Time"]]
target = "Severity1"

accident_current = accident_current[columns]
accident_current.dtypes

Road_Type                  object
Speed_limit                 int64
Light_Conditions           object
Weather_Conditions         object
Road_Surface_Conditions    object
Urban_or_Rural_Area         int64
Weekend                     int64
Night                       int64
Severity1                   int64
dtype: object

In [20]:
accident_current["Weekend"] = accident_current["Weekend"]*1
accident_current["Night"] = accident_current["Night"]*1

In [21]:
accident_current.head(5)

,Road_Type,Speed_limit,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Weekend,Night,Severity1
235930,Single carriageway,60,Darkeness: No street lighting,Fine without high winds,Dry,1,0,0,1
348561,Single carriageway,30,Daylight: Street light present,Fine without high winds,Dry,2,1,0,1
62223,Single carriageway,30,Darkness: Street lights present and lit,Fine without high winds,Dry,2,0,0,1
446182,Single carriageway,50,Daylight: Street light present,Fine without high winds,Dry,2,0,0,1
336279,Dual carriageway,40,Darkness: Street lights present and lit,Fine without high winds,Dry,1,0,0,1


In [22]:
s_df = spark.createDataFrame(accident_current)
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

s_df = s_df.na.drop()
print(s_df.dtypes)

[('Road_Type', 'string'), ('Speed_limit', 'bigint'), ('Light_Conditions', 'string'), ('Weather_Conditions', 'string'), ('Road_Surface_Conditions', 'string'), ('Urban_or_Rural_Area', 'bigint'), ('Weekend', 'bigint'), ('Night', 'bigint'), ('Severity1', 'bigint')]


In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Transformer

inputCols = ["Road_Type", "Speed_limit", "Light_Conditions", "Weather_Conditions", "Road_Surface_Conditions", "Urban_or_Rural_Area", "Weekend","Night"]
#Deal with Categorical Columns
#Transform string type columns to string indexer 
roadtypeIndexer = StringIndexer().setInputCol("Road_Type").setOutputCol("Road_TypeIndex")
speedIndexer = StringIndexer().setInputCol("Speed_limit").setOutputCol("Speed_limitIndex")
lightIndexer = StringIndexer().setInputCol("Light_Conditions").setOutputCol("Light_ConditionsIndex")
weatherIndexer = StringIndexer().setInputCol("Weather_Conditions").setOutputCol("Weather_ConditionsIndex")
roadsurfaceIndexer = StringIndexer().setInputCol("Road_Surface_Conditions").setOutputCol("Road_Surface_ConditionsIndex")
areaIndexer = StringIndexer().setInputCol("Urban_or_Rural_Area").setOutputCol("Urban_or_Rural_AreaIndex")
weekendIndexer = StringIndexer().setInputCol("Weekend").setOutputCol("WeekendIndex")
nightIndexer = StringIndexer().setInputCol("Night").setOutputCol("NightIndex")


#Transform string type columns to string indexer 
roadtypeEncoder = OneHotEncoder().setInputCol("Road_TypeIndex").setOutputCol("Road_Type_Vec")
speedEncoder = OneHotEncoder().setInputCol("Speed_limitIndex").setOutputCol("Speed_limit_Vec")
lightEncoder = OneHotEncoder().setInputCol("Light_ConditionsIndex").setOutputCol("Light_Conditions_Vec")
weatherEncoder = OneHotEncoder().setInputCol("Weather_ConditionsIndex").setOutputCol("Weather_Conditions_Vec")
roadsurfaceEncoder = OneHotEncoder().setInputCol("Road_Surface_ConditionsIndex").setOutputCol("Road_Surface_Conditions_Vec")
areaEncoder = OneHotEncoder().setInputCol("Urban_or_Rural_AreaIndex").setOutputCol("Urban_or_Rural_Area_Vec")
weekendEncoder = OneHotEncoder().setInputCol("WeekendIndex").setOutputCol("Weekend_Vec")
nightEncoder = OneHotEncoder().setInputCol("NightIndex").setOutputCol("night_Vec")


#Assemble everything together to be ("label","features") format
assembler = VectorAssembler().setInputCols(["Road_Type_Vec", "Speed_limit_Vec", "Light_Conditions_Vec", "Weather_Conditions_Vec", "Road_Surface_Conditions_Vec",
                                            "Urban_or_Rural_Area_Vec", "Weekend_Vec","night_Vec"]).setOutputCol("features")

In [24]:
# Create models
dt = DecisionTreeClassifier(labelCol='Severity1')
rf = RandomForestClassifier(labelCol="Severity1", numTrees=20)
pipeline = Pipeline().setStages([roadtypeIndexer, speedIndexer, lightIndexer, weatherIndexer, roadsurfaceIndexer,areaIndexer, weekendIndexer,nightIndexer,
                                 roadtypeEncoder, speedEncoder, lightEncoder, weatherEncoder, roadsurfaceEncoder,areaEncoder, weekendEncoder, nightEncoder, assembler])

In [25]:
# Train model. 
model_pip = pipeline.fit(s_df)
t_df = model_pip.transform(s_df)
selectedCols = ['Severity1', 'features']
df = t_df.select(selectedCols)
df.printSchema()
(trainingData, testData) = df.randomSplit([0.7, 0.3])

root
 |-- Severity1: long (nullable = true)
 |-- features: vector (nullable = true)



In [26]:
# decision tree classifier
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'Severity1', maxDepth = 3)
dtModel = dt.fit(trainingData)
dt_predictions = dtModel.transform(testData)

In [27]:
# randomforest classifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'Severity1', maxDepth = 3)
rfModel = rf.fit(trainingData)
rf_predictions = rfModel.transform(testData)

In [28]:
#decision tree
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol = 'Severity1')
print("Test Area Under ROC: " + str(evaluator.evaluate(dt_predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.6423915272610083


In [29]:
#random forest
print("Test Area Under ROC: " + str(evaluator.evaluate(rf_predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.7128516773898523


In [30]:
#logistic regression
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol = 'Severity1')

# Now we're fitting the model on a subset of data.
lrModel = lr.fit(trainingData)

# And evaluating it against the test data.
lr_predictions = lrModel.evaluate(testData)

lr_predictions.predictions.show()

+---------+--------------------+--------------------+--------------------+----------+
|Severity1|            features|       rawPrediction|         probability|prediction|
+---------+--------------------+--------------------+--------------------+----------+
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[0.69033211851941...|       0.0|
|        0|(27,[0,5,10,14,20...|[0.80167243986529...|[

In [31]:
print("Test Area Under ROC: " + str(evaluator.evaluate(lr_predictions.predictions, {evaluator.metricName: "areaUnderROC"})))

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(lrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

Test Area Under ROC: 0.7235335178629889
Multinomial coefficients: DenseMatrix([[ 0.034079  , -0.09825775, -1.54291771, -0.23551142, -1.0294194 ,
               0.40742872,  1.66335162,  1.31674229,  1.15270748,  1.28377273,
              -0.3848182 ,  0.18097202,  0.29267513, -0.34784512,  1.55568077,
               1.05723266,  1.15262672,  1.84546779,  1.91443529,  1.73338286,
              -0.20240205, -0.10707319, -0.45929637, -1.37773789, -0.43528326,
              -0.21624804, -0.44526273]])
Multinomial intercepts: [-1.114846663831627]
objectiveHistory:
0.6931354981572674
0.6596065351653655
0.6234022384054964
0.6184728569261807
0.6172958078311669
0.6146587930336769
0.6126405738737063
0.6118401033917145
0.6114733767471071
0.6112916500349317
0.6110312238966071
0.6108697593223229
0.6108158098173889
0.6107784053745544
0.6107557651935712
0.6107271916721347
0.610667899055513
0.6105644110101057
0.6104772149468729
0.6103888674072765
0.6102684774986034
0.6101989324336773
0.610028405886582

LogisticRegression_4658bae91e8bd324d4c6

In [32]:
dtModel.featureImportances

SparseVector(27, {0: 0.0291, 2: 0.0824, 5: 0.682, 10: 0.0975, 24: 0.109})

In [33]:
rfModel.featureImportances

SparseVector(27, {0: 0.0056, 1: 0.0083, 2: 0.0643, 3: 0.0005, 4: 0.0006, 5: 0.3202, 6: 0.2108, 7: 0.0054, 8: 0.0051, 9: 0.0002, 10: 0.039, 11: 0.0232, 12: 0.0771, 13: 0.0, 14: 0.0008, 15: 0.0017, 16: 0.0003, 18: 0.0002, 20: 0.0009, 22: 0.0001, 23: 0.0005, 24: 0.2151, 25: 0.0046, 26: 0.0155})

In [34]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

gbt = GBTClassifier(labelCol='Severity1',featuresCol='features')
gbt_model = gbt.fit(trainingData)

gbt_predictions = gbt_model.transform(testData)

my_binary_gbt_eval = BinaryClassificationEvaluator(labelCol='Severity1', rawPredictionCol='prediction')
print("GBT")
print(my_binary_gbt_eval.evaluate(gbt_predictions))



GBT
0.6778993149984056
